# Task 2: Bayesian Change Point Analysis

In [1]:
# import necessary libraries
import sys
import os
import pandas as pd
from glob import glob

In [2]:
from pathlib import Path
from importlib import reload
# add the project root to the path
project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

#### Data Loading

In [3]:
# Loading the module
from src.DataLoader import DataLoader
import src.DataLoader
reload(src.DataLoader)

<module 'src.DataLoader' from '/home/chalasimon/Documents/10academy/week 10/challenge/Oil-Price-Analysis/src/DataLoader.py'>

In [4]:
# data path
oil_data_path ="../data/BrentOilPrices.csv"
event_data_path = "../data/event.csv"

In [5]:
# initialize the DataLoader
oil_data_loader = DataLoader(oil_data_path)
event_data_loader = DataLoader(event_data_path)
# load the data
oil_data = oil_data_loader.load_data()
event_data = event_data_loader.load_data()

Data loaded successfully.
Data loaded successfully.
